In [3]:
import os
import pandas as pd
import datetime
import re
import logging

# Setup basic logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def list_chat_files(date_directory):
    chat_files = []
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            for team_folder in os.listdir(date_path):
                if team_folder != "KAM":
                    continue
                
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):
                                    chat_files.append(os.path.join(person_path, file))
    logging.debug(f"Chat files listed: {chat_files}")
    return chat_files

def parse_chat_file(file_path, expected_date_minus_one):
    chat_data = []
    last_person_time = None
    last_sender = None
    delay_count = 0

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            message_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*?): (.*)', line)
            system_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*)', line)
            if message_match:
                date_time_str, sender, message = message_match.groups()
            elif system_match:
                date_time_str, info = system_match.groups()
                sender = None
            else:
                continue

            date_time = pd.to_datetime(date_time_str, format='%d/%m/%y, %I:%M %p')
            if date_time.date() != expected_date_minus_one:
                continue

            is_person = sender is not None and re.match(r'^[+\d\s-]+$', sender) is None
            delay = False
            if is_person:
                if last_person_time and sender != last_sender and (date_time - last_person_time).total_seconds() > 900:
                    delay = True
                    delay_count += 1
                if not delay:
                    last_person_time = date_time
                    last_sender = sender

            chat_data.append((date_time, sender, is_person, delay))
    logging.debug(f"File parsed: {file_path}. Delays detected: {delay_count}")
    return chat_data, extract_group_name(file_path)

def create_template_dataframe():
    times = [datetime.datetime(2000, 1, 1, 0, 0) + datetime.timedelta(minutes=1 * i) for i in range(1440)]
    intervals = [time.strftime('%I:%M %p') for time in times]
    df = pd.DataFrame(index=pd.to_datetime(intervals).strftime('%I:%M %p').unique())  # Ensure unique intervals
    return df

def populate_dataframe(df, parsed_data, group_name):
    # Define new column names
    person_col = f"{group_name}_person"
    others_col = f"{group_name}_others"
    delay_col = f"{group_name}_delay"

    # Initialize new columns
    if person_col not in df.columns:
        df[person_col] = 0
    if others_col not in df.columns:
        df[others_col] = 0
    if delay_col not in df.columns:
        df[delay_col] = 0

    # Populate the new columns with parsed data
    for date_time, sender, is_person, delay in parsed_data:
        interval_index = min((date_time.hour * 60 + date_time.minute) // 1, 1439)
        interval = df.index[interval_index]

        if is_person:
            df.at[interval, person_col] = 1
        else:
            df.at[interval, others_col] = 1

        if delay:
            df.at[interval, delay_col] = 1

        logging.debug(f"Updated DataFrame at {interval} for {group_name}: Person={is_person}, Delay={delay}")

    # Update active_chat column
    if 'active_chat' not in df.columns:
        df['active_chat'] = 0

    relevant_columns = [person_col, others_col]
    df['active_chat'] = df[relevant_columns].any(axis=1).astype(int)

    return df

def extract_group_name(file_path):
    group_name = os.path.basename(file_path).replace('WhatsApp Chat with ', '').split('.')[0]
    group_name = re.sub(r'\(\d+\)$', '', group_name)  # Remove any numbers in parentheses at the end
    return group_name  # Removed the extra "_person" suffix

date_directory = "C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\drive-download-20231201T052455Z-001"
chat_files = list_chat_files(date_directory)
dataframes = {}

for file in chat_files:
    parts = file.split(os.sep)
    date_folder, person = parts[-4], parts[-2]

    try:
        folder_date = pd.to_datetime(date_folder, format='%Y-%m-%d').date()
    except ValueError:
        continue

    expected_date_minus_one = folder_date - datetime.timedelta(days=1)
    key = f"{folder_date.strftime('%Y-%m-%d')}_{person}"

    if key not in dataframes:
        dataframes[key] = create_template_dataframe()
    parsed_data, group_name = parse_chat_file(file, expected_date_minus_one)
    dataframes[key] = populate_dataframe(dataframes[key], parsed_data, group_name)
    logging.debug(f"Dataframe created for key: {key}")

# Example to show a dataframe
example_key = next(iter(dataframes))  # Just for demonstration
logging.debug(f"Example dataframe for key {example_key}: \n{dataframes[example_key]}")


2023-12-01 18:57:40,222 - DEBUG - Chat files listed: ['C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\drive-download-20231201T052455Z-001\\2023-12-01\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Aaron 2K23OCT1666R(1).txt', 'C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\drive-download-20231201T052455Z-001\\2023-12-01\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Aaron 2K23OCT1666R.txt', 'C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\drive-download-20231201T052455Z-001\\2023-12-01\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Agutha 2K23FEB1751(1).txt', 'C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\drive-download-20231201T052455Z-001\\2023-12-01\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Agutha 2K23FEB1751.txt', 'C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\drive-download-20231201T052455Z-001\\2023-12-01\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Anesu 2K22JUN1424(1).txt', 'C:\\Users\\m

2023-12-01 18:57:40,347 - DEBUG - File parsed: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Ashi_Edoofa\WhatsApp Chat with Aaron 2K23OCT1666R(1).txt. Delays detected: 0
2023-12-01 18:57:40,347 - DEBUG - Updated DataFrame at 04:59 AM for Aaron 2K23OCT1666R: Person=False, Delay=False
2023-12-01 18:57:40,351 - DEBUG - Updated DataFrame at 11:25 AM for Aaron 2K23OCT1666R: Person=True, Delay=False
2023-12-01 18:57:40,353 - DEBUG - Updated DataFrame at 11:41 AM for Aaron 2K23OCT1666R: Person=False, Delay=False
2023-12-01 18:57:40,353 - DEBUG - Updated DataFrame at 02:06 PM for Aaron 2K23OCT1666R: Person=True, Delay=False
2023-12-01 18:57:40,354 - DEBUG - Updated DataFrame at 02:11 PM for Aaron 2K23OCT1666R: Person=False, Delay=False
2023-12-01 18:57:40,355 - DEBUG - Updated DataFrame at 02:17 PM for Aaron 2K23OCT1666R: Person=True, Delay=False
2023-12-01 18:57:40,355 - DEBUG - Updated DataFrame at 02:17 PM for Aaron 2K2

2023-12-01 18:57:40,519 - DEBUG - Dataframe created for key: 2023-12-01_Ashi_Edoofa
2023-12-01 18:57:40,574 - DEBUG - File parsed: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Ashi_Edoofa\WhatsApp Chat with Agutha 2K23FEB1751.txt. Delays detected: 0
2023-12-01 18:57:40,575 - DEBUG - Updated DataFrame at 12:10 PM for Agutha 2K23FEB1751: Person=True, Delay=False
2023-12-01 18:57:40,575 - DEBUG - Updated DataFrame at 04:45 PM for Agutha 2K23FEB1751: Person=False, Delay=False
2023-12-01 18:57:40,576 - DEBUG - Updated DataFrame at 04:45 PM for Agutha 2K23FEB1751: Person=False, Delay=False
2023-12-01 18:57:40,578 - DEBUG - Dataframe created for key: 2023-12-01_Ashi_Edoofa
2023-12-01 18:57:40,614 - DEBUG - File parsed: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Ashi_Edoofa\WhatsApp Chat with Anesu 2K22JUN1424(1).txt. Delays detected: 0
2023-12-01 

2023-12-01 18:57:41,044 - DEBUG - Dataframe created for key: 2023-12-01_Ashi_Edoofa
2023-12-01 18:57:41,112 - DEBUG - File parsed: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Ashi_Edoofa\WhatsApp Chat with Anotidaishe 2K23AUG1838(1).txt. Delays detected: 0
2023-12-01 18:57:41,114 - DEBUG - Updated DataFrame at 01:15 PM for Anotidaishe 2K23AUG1838: Person=False, Delay=False
2023-12-01 18:57:41,115 - DEBUG - Updated DataFrame at 01:21 PM for Anotidaishe 2K23AUG1838: Person=True, Delay=False
2023-12-01 18:57:41,115 - DEBUG - Updated DataFrame at 01:23 PM for Anotidaishe 2K23AUG1838: Person=False, Delay=False
2023-12-01 18:57:41,116 - DEBUG - Updated DataFrame at 01:33 PM for Anotidaishe 2K23AUG1838: Person=True, Delay=False
2023-12-01 18:57:41,116 - DEBUG - Updated DataFrame at 01:38 PM for Anotidaishe 2K23AUG1838: Person=False, Delay=False
2023-12-01 18:57:41,121 - DEBUG - Dataframe created for key: 2023-12-01_Ashi

2023-12-01 18:57:41,679 - DEBUG - Updated DataFrame at 11:54 AM for Blessing 2K23SEP1692: Person=True, Delay=False
2023-12-01 18:57:41,679 - DEBUG - Updated DataFrame at 01:00 PM for Blessing 2K23SEP1692: Person=False, Delay=False
2023-12-01 18:57:41,680 - DEBUG - Updated DataFrame at 01:10 PM for Blessing 2K23SEP1692: Person=True, Delay=False
2023-12-01 18:57:41,681 - DEBUG - Updated DataFrame at 01:16 PM for Blessing 2K23SEP1692: Person=False, Delay=False
2023-12-01 18:57:41,682 - DEBUG - Updated DataFrame at 02:52 PM for Blessing 2K23SEP1692: Person=False, Delay=False
2023-12-01 18:57:41,683 - DEBUG - Updated DataFrame at 02:58 PM for Blessing 2K23SEP1692: Person=True, Delay=False
2023-12-01 18:57:41,683 - DEBUG - Updated DataFrame at 03:10 PM for Blessing 2K23SEP1692: Person=False, Delay=False
2023-12-01 18:57:41,684 - DEBUG - Updated DataFrame at 03:10 PM for Blessing 2K23SEP1692: Person=True, Delay=False
2023-12-01 18:57:41,686 - DEBUG - Dataframe created for key: 2023-12-01_Ashi

...

2023-12-01 18:57:54,151 - DEBUG - Updated DataFrame at 07:36 PM for Malcom 2K23NOV2028: Person=True, Delay=False
2023-12-01 18:57:54,152 - DEBUG - Updated DataFrame at 07:43 PM for Malcom 2K23NOV2028: Person=False, Delay=False
2023-12-01 18:57:54,152 - DEBUG - Updated DataFrame at 07:45 PM for Malcom 2K23NOV2028: Person=True, Delay=False
2023-12-01 18:57:54,153 - DEBUG - Updated DataFrame at 08:03 PM for Malcom 2K23NOV2028: Person=False, Delay=False
2023-12-01 18:57:54,154 - DEBUG - Updated DataFrame at 08:17 PM for Malcom 2K23NOV2028: Person=True, Delay=False
2023-12-01 18:57:54,154 - DEBUG - Updated DataFrame at 08:35 PM for Malcom 2K23NOV2028: Person=False, Delay=False
2023-12-01 18:57:54,154 - DEBUG - Dataframe created for key: 2023-12-01_Shivjeet Edoofa
2023-12-01 18:57:54,257 - DEBUG - File parsed: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Shivjeet Edoofa\WhatsApp Chat with Master 2K23AUG1589.txt. Delays 

2023-12-01 18:57:54,803 - DEBUG - Updated DataFrame at 05:18 AM for Prudence 2K23NOV1070R: Person=False, Delay=False
2023-12-01 18:57:54,803 - DEBUG - Updated DataFrame at 05:20 AM for Prudence 2K23NOV1070R: Person=False, Delay=False
2023-12-01 18:57:54,803 - DEBUG - Updated DataFrame at 05:41 PM for Prudence 2K23NOV1070R: Person=True, Delay=False
2023-12-01 18:57:54,817 - DEBUG - Dataframe created for key: 2023-12-01_Shivjeet Edoofa
2023-12-01 18:57:54,873 - DEBUG - File parsed: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Shivjeet Edoofa\WhatsApp Chat with Rachel 2K22NOV1502.txt. Delays detected: 0
2023-12-01 18:57:54,875 - DEBUG - Updated DataFrame at 11:28 AM for Rachel 2K22NOV1502: Person=False, Delay=False
2023-12-01 18:57:54,876 - DEBUG - Updated DataFrame at 02:27 PM for Rachel 2K22NOV1502: Person=True, Delay=False
2023-12-01 18:57:54,877 - DEBUG - Updated DataFrame at 02:28 PM for Rachel 2K22NOV1502: Pers

2023-12-01 18:57:55,148 - DEBUG - Updated DataFrame at 05:45 PM for Shepherd 2K23AUG1035: Person=True, Delay=False
2023-12-01 18:57:55,149 - DEBUG - Updated DataFrame at 06:08 PM for Shepherd 2K23AUG1035: Person=False, Delay=False
2023-12-01 18:57:55,150 - DEBUG - Updated DataFrame at 06:30 PM for Shepherd 2K23AUG1035: Person=False, Delay=False
2023-12-01 18:57:55,150 - DEBUG - Updated DataFrame at 06:31 PM for Shepherd 2K23AUG1035: Person=True, Delay=False
2023-12-01 18:57:55,151 - DEBUG - Updated DataFrame at 06:40 PM for Shepherd 2K23AUG1035: Person=False, Delay=False
2023-12-01 18:57:55,155 - DEBUG - Dataframe created for key: 2023-12-01_Shivjeet Edoofa
2023-12-01 18:57:55,170 - DEBUG - File parsed: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Shivjeet Edoofa\WhatsApp Chat with Shyleen 2K23OCT3022R.txt. Delays detected: 0
2023-12-01 18:57:55,181 - DEBUG - Updated DataFrame at 05:46 PM for Shyleen 2K23OCT3022R:

2023-12-01 18:57:55,501 - DEBUG - Updated DataFrame at 07:16 PM for Tanaka 2K22OCT2314R: Person=False, Delay=False
2023-12-01 18:57:55,501 - DEBUG - Updated DataFrame at 07:20 PM for Tanaka 2K22OCT2314R: Person=True, Delay=False
2023-12-01 18:57:55,501 - DEBUG - Dataframe created for key: 2023-12-01_Shivjeet Edoofa
2023-12-01 18:57:55,583 - DEBUG - File parsed: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Shivjeet Edoofa\WhatsApp Chat with Tanaka 2K23SEP1123.txt. Delays detected: 0
2023-12-01 18:57:55,586 - DEBUG - Updated DataFrame at 12:48 PM for Tanaka 2K23SEP1123: Person=True, Delay=False
2023-12-01 18:57:55,586 - DEBUG - Updated DataFrame at 02:57 PM for Tanaka 2K23SEP1123: Person=True, Delay=False
2023-12-01 18:57:55,587 - DEBUG - Updated DataFrame at 08:42 PM for Tanaka 2K23SEP1123: Person=False, Delay=False
2023-12-01 18:57:55,587 - DEBUG - Updated DataFrame at 09:22 PM for Tanaka 2K23SEP1123: Person=True,

2023-12-01 18:57:55,857 - DEBUG - Dataframe created for key: 2023-12-01_Shivjeet Edoofa
2023-12-01 18:57:55,876 - DEBUG - File parsed: C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\drive-download-20231201T052455Z-001\2023-12-01\KAM\Shivjeet Edoofa\WhatsApp Chat with Tinevimbo 2K23NOV1559.txt. Delays detected: 0
2023-12-01 18:57:55,881 - DEBUG - Updated DataFrame at 04:27 PM for Tinevimbo 2K23NOV1559: Person=True, Delay=False
2023-12-01 18:57:55,884 - DEBUG - Updated DataFrame at 05:00 PM for Tinevimbo 2K23NOV1559: Person=False, Delay=False
2023-12-01 18:57:55,885 - DEBUG - Updated DataFrame at 05:00 PM for Tinevimbo 2K23NOV1559: Person=False, Delay=False
2023-12-01 18:57:55,887 - DEBUG - Updated DataFrame at 05:11 PM for Tinevimbo 2K23NOV1559: Person=True, Delay=False
2023-12-01 18:57:55,889 - DEBUG - Updated DataFrame at 05:50 PM for Tinevimbo 2K23NOV1559: Person=False, Delay=False
2023-12-01 18:57:55,892 - DEBUG - Updated DataFrame at 05:50 PM for Tinevimbo 2K23N